<a href="https://colab.research.google.com/github/jvataidee/MachineLearning/blob/master/Churn_Prediction_para_uma_empresa_de_Telecomunica%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p align="center"><img src="https://static.wixstatic.com/media/bdd7cb_6ecb6c85550043f090237e5943d2647c~mv2.png/v1/fill/w_417,h_88,al_c,q_85,usm_0.66_1.00_0.01/Ativo%2023-8.webp" height="75px"></p>

**by: [João Ataíde](https://www.joaoataide.com)**
#**Predição de Evasão de Clientes em Empresas de Telecomunicações**


---

<p align="center"><img src="https://static.wixstatic.com/media/bdd7cb_a558eaed78994bcb94f2c0e83f5722ff~mv2.jpg"></p>


Empresas prestadoras de serviços comunicação costumam usar análises de insatisfação de cliente, identificando sempre se seus clientes estão confirmados ou não, essas análises são umas das principais métricas para manter e adquirir novos compradores. Em geral, as empresas desses setores costumam ter agências de atendimento ao cliente, onde uma das suas principais tarefas é tentar reconquistar aqueles clientes perdidos, os quais podem custar muito mais do que clientes novos.

Devido a isso, as empresas investem em nova tecnologias para fazer essas análises, estas comummente utilizam-se do *Churn rate*, ou simplesmente *Churn*, que é uma das formas de medir essa taxa de evasão dos clientes, 
Além de identificar essa taxa de evasão, o *Churn* auxiliar a identificar futuros cancelamento, realizando uma predição que auxilia na tomada de decisão e promoção de ações diferenciadas para estes clientes. Uma aplicação cotidiana nossa são nas empresas de streaming como Spotifly e Netflix. 

Por ser uma métrica tão importante, o Carlos Melos no seu curso Data Science na prática nos passou um desafio, implementar do zero uma solução de Churn para uma empresa de telecomunicação, utilizando-se de dados reais.


## Aquisição dos Dados e importação das bibliotecas



Os dados utilizados neste projeto foram originalmente disponibilizados na [plataforma de ensino da IBM Developer](https://developer.ibm.com/technologies/data-science/patterns/predict-customer-churn-using-watson-studio-and-jupyter-notebooks/#), e tratam de um problema típico de uma companhia de telecomunicações. O *dataset* completo pode ser encontrado [neste link](https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv).

Apesar de não haver informações explícitas disponíves, os nomes das colunas permitem um entendimento a respeito do problema.

**Falar sobre os dados**

In [4]:
#importar bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Bibliotecas de modelagem


# configurar a visualização
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# importar os dados
df = "https://raw.githubusercontent.com/jvataidee/MachineLearning/master/dados/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(df)

# ver as 5 primeiras entradas

print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


A visualização dessas 5 primeiras entradas permite a notar a composição do nosso *dataset*, mesmo que este não apresente uma definição oficial de cada coluna. O *dataset* apresenta 7042 entradas com 21 colunas.

## Análise Exploratória 
 

Como forma de avalisar a qualidade do *dataset*, comecei pela análise dos dados ausentes, este que indica se poderemos dá continuidade ao projeto ou não




In [10]:
#Porcentagem de dados ausentes
df.isnull().sum()/df.shape[0]*100

customerID          0.0
gender              0.0
SeniorCitizen       0.0
Partner             0.0
Dependents          0.0
tenure              0.0
PhoneService        0.0
MultipleLines       0.0
InternetService     0.0
OnlineSecurity      0.0
OnlineBackup        0.0
DeviceProtection    0.0
TechSupport         0.0
StreamingTV         0.0
StreamingMovies     0.0
Contract            0.0
PaperlessBilling    0.0
PaymentMethod       0.0
MonthlyCharges      0.0
TotalCharges        0.0
Churn               0.0
dtype: float64

Após o calculo podemos ver que esse *dataset* já está limpo, nçao apresentando nenhum dados ausente. O proximo passo ´re calcular dentre essas culunas quais apresentam valores únicos.

In [11]:
#Calcular valores únicos
df.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

O cálculo dos valores únicos indicaram que, a coluna  `customerID`apresenta um valor unico para cada cliente, `Churn`indica a situação de cada cliente. Proximo passo irei verifiar qual a situação dos dados em relação ao tipo.


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Veja que a camada  ` TotalCharges`está apresentando um tipo object (*String*), desta forma irei converter para *Float*, pois se trata de um valor não inteiro.

In [21]:
#Converter para float
df.TotalCharges = pd.to_numeric(df.TotalCharges,errors='coerce')

Agora, irei realizar uma copia do *dataset*, descartar a coluna ` customerID ` e então da inicio a estatistica descritiva.

In [28]:
#Conpiar dados
df_new = df.copy()

df_new = df_new.drop(['customerID'], axis = 1)

In [30]:
df_new.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


boxplot
histogramas

###Balanceamento dos dados

##Soluções já existentes

##Construção do modelo

###Métrica de peformace

##Qual melhor modelo?

###Preparação dos dados

####Cross Validation

##Conclusão